In [2]:
from pulse2percept.models import BiphasicAxonMapModel, AxonMapModel
from pulse2percept.model_selection import ParticleSwarmOptimizer
from pulse2percept.implants import ArgusII
import pulse2percept as p2p
import shapes
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [5]:
s2 = shapes.subject_params['12-005']
implant, model = shapes.model_from_params(s2)
model.xystep=0.5

estimator = shapes.BiphasicAxonMapEstimator(implant=implant, model=model,resize=True, threshold='compute', loss_fn='r2',mse_params=['area','orientation','eccentricity'])
search_params = {
    'rho': (10, 1000),
    'axlambda': (10, 2000)
}
opt = ParticleSwarmOptimizer(estimator, search_params, max_iter=100, swarm_size=100, has_loss_function=True)

data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005'], stim_class='SingleElectrode')
x = data[['amp1', 'freq', 'pdur', 'electrode1']].reset_index(drop=True)
y = data['image'].reset_index(drop=True)
y_moments = estimator.compute_moments(y)
a = data[['electrode1','freq','amp1','pdur']].value_counts().reset_index().iloc[: , :-1]
moments_avg = []

for i in range(len(a)):
    temp_x = x[(x['electrode1'] == a['electrode1'][i]) & (x['freq'] == a['freq'][i]) & (x['amp1'] == a['amp1'][i])]
    y_moments_temp = y_moments[temp_x.index.tolist()[0]]
    for j in temp_x.index.tolist()[1:]:
        y_moments_temp = y_moments_temp + y_moments[j]
    moments_avg.append(np.array(y_moments_temp/len(temp_x)))

train_mse = []
test_mse = []
train_size = []
test_size = []
estimator_params = []
filename_lst = []

# fitting and finding the best parameter
best_mse = 0
best_score = 99999
best_estimator = estimator
for itr in range(3):
    opt = ParticleSwarmOptimizer(estimator, search_params, max_iter=100, swarm_size=100, has_loss_function=True)
    estimator.fit_size_model(x['amp1'], y)
    opt.fit(a, np.array(moments_avg))
    score = estimator.score(a, np.array(moments_avg),return_mses=True)
    if score[0]<best_score: 
        best_score = score[0]
        best_estimator = estimator
        best_mse = score[1]

# store train results and estimator
train_mse.append([best_score,best_mse])
estimator = best_estimator
estimator_params.append(estimator.get_params())

# test (recompute y moments)
estimator.mse_params = ["area", "eccentricity", "major_axis_length", "minor_axis_length", "orientation"]
estimator.scale_features=True
y = data['image'].reset_index(drop=True)
new_y_moments = estimator.compute_moments(y, fit_scaler=True)
new_moments_avg = []
for i in range(len(a)):
    temp_x = x[(x['electrode1'] == a['electrode1'][i]) & (x['freq'] == a['freq'][i]) & (x['amp1'] == a['amp1'][i])]
    y_moments_temp = new_y_moments[temp_x.index.tolist()[0]]
    for j in temp_x.index.tolist()[1:]:
        y_moments_temp = y_moments_temp + new_y_moments[j]
    new_moments_avg.append(np.array(y_moments_temp/len(temp_x)))


# store test results
test_size.append(len(a))
test_mse.append(estimator.score(a, np.array(new_moments_avg),return_mses=True))

rho = []
axlambda = []
a5 = []
train_MSE_combined = []
train_MSE_area = []
train_MSE_orientation = []
train_MSE_eccentricity = []

test_MSE_combined = []
test_MSE_area = []
test_MSE_eccentricity = []
test_MSE_major_axis_length = []
test_MSE_minor_axis_length = []
test_MSE_orientation = []


rho.append(estimator_params[-1]['rho'])
axlambda.append(estimator_params[-1]['axlambda'])
a5.append(estimator_params[-1]['a5'])
train_MSE_combined.append(train_mse[-1][0])
train_MSE_area.append(train_mse[-1][1][0])
        
test_MSE_combined.append(test_mse[-1][0])


test_MSE_area.append(test_mse[-1][1][0])
test_MSE_eccentricity.append(test_mse[-1][1][1])    
test_MSE_major_axis_length.append(test_mse[-1][1][2])
test_MSE_minor_axis_length.append(test_mse[-1][1][3])
test_MSE_orientation.append(test_mse[-1][1][4])

df = pd.DataFrame({
                       'rho':rho,
                       'axlambda':axlambda,
                       'a5':a5,
                       'train_combined':train_MSE_combined,
                       'train_area':train_MSE_area,
                       'test_combined':test_MSE_combined,
                       'test_area':test_MSE_area,
                       'test_eccentricity':test_MSE_eccentricity,
                       'test_major_axis_length':test_MSE_major_axis_length,
                       'test_minor_axis_length':test_MSE_minor_axis_length,
                       'test_orientation':test_MSE_orientation})
df.to_csv('biphasic_s2_compute'+ '.csv', encoding='utf-8')

Removing means (['area: 560.18', 'orientation: 0.06', 'eccentricity: 0.87']) 
Scaling standard deviations (['area: 392.19', 'orientation: 0.94', 'eccentricity: 0.15']) to be 1
a5=0.185353, a6=0.629293
score:1004.343, rho:644.0, lambda:1939.5, a5:0.185, empty:0, scores: ['area:997.7', 'orientation:2.1', 'eccentricity:4.5']
score:271.919, rho:474.5, lambda:1178.3, a5:0.185, empty:0, scores: ['area:264.7', 'orientation:2.3', 'eccentricity:4.9']
score:67.899, rho:303.7, lambda:1076.7, a5:0.185, empty:0, scores: ['area:62.9', 'orientation:1.8', 'eccentricity:3.2']
score:29.179, rho:302.1, lambda:231.6, a5:0.185, empty:0, scores: ['area:10.7', 'orientation:1.8', 'eccentricity:16.6']
score:227.355, rho:491.1, lambda:851.7, a5:0.185, empty:0, scores: ['area:218.8', 'orientation:2.0', 'eccentricity:6.6']
score:13.748, rho:133.1, lambda:1417.9, a5:0.185, empty:0, scores: ['area:10.1', 'orientation:1.2', 'eccentricity:2.5']
score:357.236, rho:446.7, lambda:1871.7, a5:0.185, empty:0, scores: ['are

In [15]:
s3 = shapes.subject_params['51-009']
implant, model = shapes.model_from_params(s3)
model.xystep=0.5

estimator = shapes.BiphasicAxonMapEstimator(implant=implant, model=model,resize=True, threshold='compute', loss_fn='r2',mse_params=['area','orientation','eccentricity'])
search_params = {
    'rho': (10, 1000),
    'axlambda': (10, 2000)
}
opt = ParticleSwarmOptimizer(estimator, search_params, max_iter=100, swarm_size=100, has_loss_function=True)

data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['51-009'], stim_class='SingleElectrode')
x = data[['amp1', 'freq', 'pdur', 'electrode1']].reset_index(drop=True)
y = data['image'].reset_index(drop=True)
y_moments = estimator.compute_moments(y)
a = data[['electrode1','freq','amp1','pdur']].value_counts().reset_index().iloc[: , :-1]
moments_avg = []

for i in range(len(a)):
    temp_x = x[(x['electrode1'] == a['electrode1'][i]) & (x['freq'] == a['freq'][i]) & (x['amp1'] == a['amp1'][i])]
    y_moments_temp = y_moments[temp_x.index.tolist()[0]]
    for j in temp_x.index.tolist()[1:]:
        y_moments_temp = y_moments_temp + y_moments[j]
    moments_avg.append(np.array(y_moments_temp/len(temp_x)))

train_mse = []
test_mse = []
train_size = []
test_size = []
estimator_params = []
filename_lst = []

# fitting and finding the best parameter
best_mse = 0
best_score = 99999
best_estimator = estimator
for itr in range(3):
    opt = ParticleSwarmOptimizer(estimator, search_params, max_iter=100, swarm_size=100, has_loss_function=True)
    estimator.fit_size_model(x['amp1'], y)
    opt.fit(a, np.array(moments_avg))
    score = estimator.score(a, np.array(moments_avg),return_mses=True)
    if score[0]<best_score: 
        best_score = score[0]
        best_estimator = estimator
        best_mse = score[1]

# store train results and estimator
train_mse.append([best_score,best_mse])
estimator = best_estimator
estimator_params.append(estimator.get_params())

# test (recompute y moments)
estimator.mse_params = ["area", "eccentricity", "major_axis_length", "minor_axis_length", "orientation"]
estimator.scale_features=True
y = data['image'].reset_index(drop=True)
new_y_moments = estimator.compute_moments(y, fit_scaler=True)
new_moments_avg = []
for i in range(len(a)):
    temp_x = x[(x['electrode1'] == a['electrode1'][i]) & (x['freq'] == a['freq'][i]) & (x['amp1'] == a['amp1'][i])]
    y_moments_temp = new_y_moments[temp_x.index.tolist()[0]]
    for j in temp_x.index.tolist()[1:]:
        y_moments_temp = y_moments_temp + new_y_moments[j]
    new_moments_avg.append(np.array(y_moments_temp/len(temp_x)))


# store test results
test_size.append(len(a))
test_mse.append(estimator.score(a, np.array(new_moments_avg),return_mses=True))

rho = []
axlambda = []
a5 = []
train_MSE_combined = []
train_MSE_area = []
train_MSE_orientation = []
train_MSE_eccentricity = []

test_MSE_combined = []
test_MSE_area = []
test_MSE_eccentricity = []
test_MSE_major_axis_length = []
test_MSE_minor_axis_length = []
test_MSE_orientation = []


rho.append(estimator_params[-1]['rho'])
axlambda.append(estimator_params[-1]['axlambda'])
a5.append(estimator_params[-1]['a5'])
train_MSE_combined.append(train_mse[-1][0])
train_MSE_area.append(train_mse[-1][1][0])
        
test_MSE_combined.append(test_mse[-1][0])


test_MSE_area.append(test_mse[-1][1][0])
test_MSE_eccentricity.append(test_mse[-1][1][1])    
test_MSE_major_axis_length.append(test_mse[-1][1][2])
test_MSE_minor_axis_length.append(test_mse[-1][1][3])
test_MSE_orientation.append(test_mse[-1][1][4])

df = pd.DataFrame({
                       'rho':rho,
                       'axlambda':axlambda,
                       'a5':a5,
                       'train_combined':train_MSE_combined,
                       'train_area':train_MSE_area,
                       'test_combined':test_MSE_combined,
                       'test_area':test_MSE_area,
                       'test_eccentricity':test_MSE_eccentricity,
                       'test_major_axis_length':test_MSE_major_axis_length,
                       'test_minor_axis_length':test_MSE_minor_axis_length,
                       'test_orientation':test_MSE_orientation})
df.to_csv('biphasic_s3_compute'+ '.csv', encoding='utf-8')

Removing means (['area: 170.76', 'orientation: -0.21', 'eccentricity: 0.98']) 
Scaling standard deviations (['area: 103.78', 'orientation: 0.81', 'eccentricity: 0.07']) to be 1
a5=-0.025634, a6=1.051269
score:417.387, rho:187.4, lambda:1893.5, a5:-0.026, empty:0, scores: ['area:367.4', 'orientation:2.3', 'eccentricity:47.7']
score:2647.249, rho:485.9, lambda:1183.2, a5:-0.026, empty:0, scores: ['area:2523.8', 'orientation:4.6', 'eccentricity:118.8']
score:2360.226, rho:431.4, lambda:1519.5, a5:-0.026, empty:0, scores: ['area:2263.7', 'orientation:3.7', 'eccentricity:92.8']
score:2045.110, rho:527.0, lambda:342.9, a5:-0.026, empty:0, scores: ['area:1595.6', 'orientation:4.4', 'eccentricity:445.1']
score:107.352, rho:169.8, lambda:631.9, a5:-0.026, empty:0, scores: ['area:60.4', 'orientation:2.4', 'eccentricity:44.6']
score:16336.545, rho:875.9, lambda:1248.3, a5:-0.026, empty:0, scores: ['area:16102.8', 'orientation:6.3', 'eccentricity:227.4']
score:24149.660, rho:977.7, lambda:1319.9, 

In [16]:
s4 = shapes.subject_params['52-001']
implant, model = shapes.model_from_params(s4)
model.xystep=0.5

estimator = shapes.BiphasicAxonMapEstimator(implant=implant, model=model,resize=True, threshold='compute', loss_fn='r2',mse_params=['area','orientation','eccentricity'])
search_params = {
    'rho': (10, 1000),
    'axlambda': (10, 2000)
}
opt = ParticleSwarmOptimizer(estimator, search_params, max_iter=100, swarm_size=100, has_loss_function=True)

data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['52-001'], stim_class='SingleElectrode')
x = data[['amp1', 'freq', 'pdur', 'electrode1']].reset_index(drop=True)
y = data['image'].reset_index(drop=True)
y_moments = estimator.compute_moments(y)
a = data[['electrode1','freq','amp1','pdur']].value_counts().reset_index().iloc[: , :-1]
moments_avg = []

for i in range(len(a)):
    temp_x = x[(x['electrode1'] == a['electrode1'][i]) & (x['freq'] == a['freq'][i]) & (x['amp1'] == a['amp1'][i])]
    y_moments_temp = y_moments[temp_x.index.tolist()[0]]
    for j in temp_x.index.tolist()[1:]:
        y_moments_temp = y_moments_temp + y_moments[j]
    moments_avg.append(np.array(y_moments_temp/len(temp_x)))

train_mse = []
test_mse = []
train_size = []
test_size = []
estimator_params = []
filename_lst = []

# fitting and finding the best parameter
best_mse = 0
best_score = 99999
best_estimator = estimator
for itr in range(3):
    opt = ParticleSwarmOptimizer(estimator, search_params, max_iter=100, swarm_size=100, has_loss_function=True)
    estimator.fit_size_model(x['amp1'], y)
    opt.fit(a, np.array(moments_avg))
    score = estimator.score(a, np.array(moments_avg),return_mses=True)
    if score[0]<best_score: 
        best_score = score[0]
        best_estimator = estimator
        best_mse = score[1]

# store train results and estimator
train_mse.append([best_score,best_mse])
estimator = best_estimator
estimator_params.append(estimator.get_params())

# test (recompute y moments)
estimator.mse_params = ["area", "eccentricity", "major_axis_length", "minor_axis_length", "orientation"]
estimator.scale_features=True
y = data['image'].reset_index(drop=True)
new_y_moments = estimator.compute_moments(y, fit_scaler=True)
new_moments_avg = []
for i in range(len(a)):
    temp_x = x[(x['electrode1'] == a['electrode1'][i]) & (x['freq'] == a['freq'][i]) & (x['amp1'] == a['amp1'][i])]
    y_moments_temp = new_y_moments[temp_x.index.tolist()[0]]
    for j in temp_x.index.tolist()[1:]:
        y_moments_temp = y_moments_temp + new_y_moments[j]
    new_moments_avg.append(np.array(y_moments_temp/len(temp_x)))


# store test results
test_size.append(len(a))
test_mse.append(estimator.score(a, np.array(new_moments_avg),return_mses=True))

rho = []
axlambda = []
a5 = []
train_MSE_combined = []
train_MSE_area = []
train_MSE_orientation = []
train_MSE_eccentricity = []

test_MSE_combined = []
test_MSE_area = []
test_MSE_eccentricity = []
test_MSE_major_axis_length = []
test_MSE_minor_axis_length = []
test_MSE_orientation = []


rho.append(estimator_params[-1]['rho'])
axlambda.append(estimator_params[-1]['axlambda'])
a5.append(estimator_params[-1]['a5'])
train_MSE_combined.append(train_mse[-1][0])
train_MSE_area.append(train_mse[-1][1][0])
        
test_MSE_combined.append(test_mse[-1][0])


test_MSE_area.append(test_mse[-1][1][0])
test_MSE_eccentricity.append(test_mse[-1][1][1])    
test_MSE_major_axis_length.append(test_mse[-1][1][2])
test_MSE_minor_axis_length.append(test_mse[-1][1][3])
test_MSE_orientation.append(test_mse[-1][1][4])

df = pd.DataFrame({
                       'rho':rho,
                       'axlambda':axlambda,
                       'a5':a5,
                       'train_combined':train_MSE_combined,
                       'train_area':train_MSE_area,
                       'test_combined':test_MSE_combined,
                       'test_area':test_MSE_area,
                       'test_eccentricity':test_MSE_eccentricity,
                       'test_major_axis_length':test_MSE_major_axis_length,
                       'test_minor_axis_length':test_MSE_minor_axis_length,
                       'test_orientation':test_MSE_orientation})
df.to_csv('biphasic_s4_compute'+ '.csv', encoding='utf-8')

Removing means (['area: 1060.64', 'orientation: 0.04', 'eccentricity: 0.88']) 
Scaling standard deviations (['area: 1296.6', 'orientation: 0.79', 'eccentricity: 0.16']) to be 1
a5=1.382470, a6=-1.764940
score:75.490, rho:902.8, lambda:1733.5, a5:1.382, empty:0, scores: ['area:65.0', 'orientation:4.6', 'eccentricity:5.9']
score:8.056, rho:385.5, lambda:1442.6, a5:1.382, empty:0, scores: ['area:3.0', 'orientation:2.8', 'eccentricity:2.3']
score:27.550, rho:576.9, lambda:368.1, a5:1.382, empty:0, scores: ['area:4.7', 'orientation:3.1', 'eccentricity:19.8']
score:4.420, rho:138.5, lambda:408.5, a5:1.382, empty:0, scores: ['area:1.3', 'orientation:1.5', 'eccentricity:1.6']
score:11.778, rho:444.1, lambda:1783.4, a5:1.382, empty:0, scores: ['area:6.3', 'orientation:2.9', 'eccentricity:2.5']
score:4.907, rho:277.1, lambda:1507.1, a5:1.382, empty:0, scores: ['area:1.2', 'orientation:1.9', 'eccentricity:1.8']
score:11.890, rho:456.1, lambda:1661.6, a5:1.382, empty:0, scores: ['area:6.3', 'orien